In [21]:
#function to write in csv
import csv
def write_list_in_file(final, name):
    with open(name, "w", newline="",encoding="utf8") as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(final)


In [22]:
#Function to read csv files
from csv import reader
# Load a CSV file\n",
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


In [23]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#load base data
base_data = load_csv('season1.csv')    
base_data = np.array(base_data[1:])
base_size = len(base_data)

#no of target elements and load target data
elements = [10,20,30,40,50]
target_data = load_csv('season2.csv')
target_data = np.array(target_data[1:])

#normalize data using a min max normalizer
mixed = np.concatenate((base_data,target_data))
scaler = MinMaxScaler()
mixed_scaled = scaler.fit_transform(mixed)

#normalized base and target data
base_scaled = mixed_scaled[:base_size]
target_scaled = mixed_scaled[base_size:]


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype <U9 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [46]:
# number of attributes
num_attr = mixed_scaled[0].size

In [47]:
#learn number of clusters using silhouette_score

from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

num_clusters = 0
max_silhouette = -1

for n_clusters in range(2,10):

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(base_scaled)

    silhouette_avg = silhouette_score(base_scaled, cluster_labels)
    #print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)
    
    if(silhouette_avg > max_silhouette):
        max_silhouette = silhouette_avg
        num_clusters = n_clusters

print("Optimum Number of Clusters: ",num_clusters)


Optimum Number of Clusters:  7


In [48]:
#learn centers from source/base model
import numpy as np
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(base_scaled)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
#print(kmeans.cluster_centers_)
#print(kmeans.inertia_)

In [52]:
#base model results
kmeans = KMeans(n_clusters=num_clusters, init=base_centroids, max_iter=1)
kmeans.fit(target_scaled)
print("applying base model in target result: ", kmeans.inertia_)

applying base model in target result:  7.99637305263


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [53]:
#retraining results


for n in elements:

    #Read Available Limited Target Data
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(target_scaled[:n])
    
    #Result on whole Target Data
    kmeans = KMeans(n_clusters=num_clusters, init=kmeans.cluster_centers_, max_iter=1)
    kmeans.fit(target_scaled)

    print("Retraining results on ", n," data ",kmeans.inertia_)

Retraining results on  10  data  9.63067393178
Retraining results on  20  data  9.25081005174
Retraining results on  30  data  6.45792655246
Retraining results on  40  data  6.00573365415
Retraining results on  50  data  5.92548512222


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [11]:
#reframing